In [6]:
# chatbot
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# llm
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

import os
import gradio as gr
import os

api_key ='내 키 올리기' 


보안유지 ( api_key는 유출상, github 올리지 말기)

In [8]:
import openai
openai.api_key = api_key
audio_file = open("./dd.mp3", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)
text = transcript['text']
print(text)

에요. 네. 일단 잘 알겠고요. 저희들이 kb금융괴물사 kb제출은행 인데요. 지금 5월달에 저희들이 대환대출 플러스 생활루지자금 쪽으로 나온 대출 상품이 있어요. 그리고 연이후리 7.8%대로 진행하는 부분이고 상환기간이 5년까지고요 . 중도상환 가능하고 수수료 발생 없이 원리금균동분할상환 혹은 만길수 상환 쪽으로 이자만 갚아도 되는 부분이에요. 그리고 최대 맥심 대환대출 플러스 생활루지자금 쪽으로 5천만원까지 인데 5천만원 금도 다 받으실 거예요 . 일단 잘 알겠고요. 정... 맞으시죠. 네. 고객님 혹시 지금 사용하고 있는 휴대폰은 본인 이름으로 되어 있는 거 맞으세요 통신사는 skt에요 lg 에요 삼성 스마트폰 맞으세요. 혹시 카톡은 사용하고 계세요 네 그러면 카톡 추가를 해서 저희들 회사 상문하고 제 이름도 드릴 게 요. 왜요 예 왜요 고객님 저한테 전화를 주셔서 통화 중에 제가 전화를 못 받으면 카톡 으로 연락을 할 수도 있어서 그런 거 아니에요. 아니요. 전화로 주세요. 네 전화로 주시면 되죠.


In [9]:
import pandas as pd

data = pd.read_csv("data.csv")
data

,이름,주민등록상,문제
0,김태경,혼자,건강보험료 연체
1,주예서,혼자,건강 문제
2,강민지,2,공과급 체납


순서도

1. 가구

2. 건강

3. 연체

4. 추가 상담 여부

In [10]:
chat_template = f"""필수 : 나는 복지상담을 진행하는 음성봇 이며 이름은 아이이다.
한번의 답변에 한가지 질문만 한다. 
공손하고 예의바르게 말을 해야한다. 
전화통화를 하는 것처럼 대화를 해야한다. 
답변은 2줄이내로 진행한다. 
해당 전화는 본 챗봇이 먼저 전화를 건 상황이다.
총 6번이상의 대화가 오고가도록 한다

0. 상대방의 이름은 {data["이름"][1]}이다. 해당 이름으로 부르며 대화를 한다.

1. 첫번째 답변은 안녕하세요 000님, 지금 현재 {data["문제"][1]}로 확인이 되는데 어려움은 없는지 질문한다.

2. {data["문제"][1]}이 건강문제라면 어디 아프신곳이 있냐고 질문한다.

3. {data["문제"][1]}에 대해 구체적인 상황은 어떠한지 질문을 한다.

4. 상대방의 문제에 대해 어느정도 질문을 했으면 주변에 도움을 요청할만한 보호자가 없는지 물어본다.

5. 질문이 최소 5번 이상 오가고 난 이후에, 상대방의 어려움에 관련된 복지 서비스를 추천해준다.


"""

chat_ai  = ChatOpenAI(openai_api_key=api_key,temperature=0.5, model="gpt-4-1106-preview")

def response(message, history):  ## human, ai 리스트 반환으로 지정
    # 종료하겠습니다를 입력하면 최종 요약 출력
    if message == "종료하겠습니다":
        return summarize(history)

    history_langchain_format = []
    # 프롬프트 추가
    history_langchain_format.append(SystemMessage(content=chat_template))
    # 이전 대화 기억
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = chat_ai(history_langchain_format)
    return gpt_response.content


# 요약 llm 모델 추가
# summarize_llm과 history로 출력
def summarize(history):
    for i, (human, ai) in enumerate(history):
        history[i] = (f"사용자 : {human}", f"AI : {ai}")

    summarize_llm = OpenAI(
        temperature=0.3, model="gpt-3.5-turbo-instruct", max_tokens=512
    )


    summarize_template = """
    필수 : AI와 사용자 모두의 대화를 합쳐서 요약할거야.
    0. 사용자의 문제상황이 뭔지 정확하게 파악하고 요약해야해.

    {texts} 대화내용을 기반으로 요약문 작성해줘.
    """

    summarize_prompt = PromptTemplate(
        template=summarize_template, input_variables=["texts"]
    )
    summarize_chain = LLMChain(prompt=summarize_prompt, llm=summarize_llm)

    return summarize_chain.run(history)


gr.ChatInterface(
    fn=response,
    textbox=gr.Textbox(placeholder="말걸어주세요..", container=False, scale=7),
    # 채팅창의 크기를 조절한다.
    chatbot=gr.Chatbot(height=1000),
    title="아이(윙윙이)",
    description='대화가 끝나면 "대화종료" 를 입력해주세요',
    theme="soft",
    retry_btn="다시보내기 ↩",
    undo_btn="이전챗 삭제 ❌",
    clear_btn="전챗 삭제 💫",
).launch(share=True)

Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
